In [3]:
!pip install numpy trimesh tetgen plotly polyscope pyvista meshio

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import numpy as np
import trimesh
import tetgen
import plotly.graph_objects as go
import polyscope as ps
import pyvista as pv
import meshio
import sys

# -------------------------------------------
# 1. Check Trimesh Version
# -------------------------------------------
required_trimesh_version = "3.8.9"

def check_trimesh_version(required_version):
    current_version = trimesh.__version__
    from packaging import version
    if version.parse(current_version) < version.parse(required_version):
        print(f"Trimesh version {current_version} detected. Version {required_version} or newer is required.")
        print("Please update Trimesh using 'pip install --upgrade trimesh' and rerun the script.")
        sys.exit(1)
    else:
        print(f"Trimesh version {current_version} is sufficient.")

check_trimesh_version(required_trimesh_version)

# -------------------------------------------
# 2. Initialize Polyscope
# -------------------------------------------
ps.init()

# -------------------------------------------
# 3. Define Surface Parameters and Generate Rough Surface
# -------------------------------------------
width, height = 100, 100  # Size of the 2D surface grid in µm
Ra = 1.0  # Average roughness in µm
Rq = 1.5  # Root mean square roughness in µm

# Generate a rough surface using Gaussian random heights
np.random.seed(42)  # For reproducibility
surface_height = np.random.normal(loc=Ra, scale=Rq, size=(width, height))

# Generate X and Y grid for the 3D surface plot
X, Y = np.meshgrid(np.arange(width), np.arange(height))
Z = surface_height

# Flatten the grid for Mesh3d and Trimesh
x_flat = X.flatten()
y_flat = Y.flatten()
z_flat = Z.flatten()

# Create the main surface mesh using Trimesh
vertices = np.vstack((x_flat, y_flat, z_flat)).T

# Function to generate faces for the main surface
def create_surface_faces(width, height):
    faces = []
    for i in range(width - 1):
        for j in range(height - 1):
            # Calculate the indices of the square's corners
            idx = i * height + j
            idx_right = (i + 1) * height + j
            idx_up = i * height + (j + 1)
            idx_right_up = (i + 1) * height + (j + 1)
            
            # First triangle
            faces.append([idx, idx_right, idx_up])
            # Second triangle
            faces.append([idx_right, idx_right_up, idx_up])
    return faces

# Generate faces for the main surface
surface_faces = create_surface_faces(width, height)

# Create the base vertices (z=0)
base_vertices = np.vstack((x_flat, y_flat, np.zeros_like(z_flat))).T

# Create faces for the base
base_faces = create_surface_faces(width, height)

# Offset base face indices
base_faces = np.array(base_faces) + len(vertices)

# Combine main surface and base
all_vertices = np.vstack((vertices, base_vertices))
all_faces = np.vstack((surface_faces, base_faces))

# -------------------------------------------
# 4. Create Side Walls to Make Mesh Watertight
# -------------------------------------------
def create_side_faces(width, height):
    walls = []
    
    # Front and Back Walls (y=0 and y=height-1)
    for i in range(width -1):
        # Front wall (y=0)
        idx0 = i * height
        idx1 = (i +1) * height
        idx0_base = idx0 + width * height
        idx1_base = idx1 + width * height
        walls.append([idx0, idx1, idx1_base])
        walls.append([idx0, idx1_base, idx0_base])
        
        # Back wall (y=height-1)
        idx0 = i * height + (height -1)
        idx1 = (i +1) * height + (height -1)
        idx0_base = idx0 + width * height
        idx1_base = idx1 + width * height
        walls.append([idx0, idx1, idx1_base])
        walls.append([idx0, idx1_base, idx0_base])
    
    # Left and Right Walls (x=0 and x=width-1)
    for j in range(height -1):
        # Left wall (x=0)
        idx0 = j
        idx1 = j +1
        idx0_base = idx0 + width * height
        idx1_base = idx1 + width * height
        walls.append([idx0, idx1, idx1_base])
        walls.append([idx0, idx1_base, idx0_base])
        
        # Right wall (x=width-1)
        idx0 = (width -1) * height + j
        idx1 = (width -1) * height + (j +1)
        idx0_base = idx0 + width * height
        idx1_base = idx1 + width * height
        walls.append([idx0, idx1, idx1_base])
        walls.append([idx0, idx1_base, idx0_base])
    
    return walls

# Generate side walls
side_faces = create_side_faces(width, height)

# Combine all faces
all_faces = np.vstack((all_faces, side_faces))

# -------------------------------------------
# 5. Create and Repair Trimesh Object
# -------------------------------------------
# Create a Trimesh object
mesh = trimesh.Trimesh(vertices=all_vertices, faces=all_faces, process=True)

# Function to diagnose and repair the mesh
def diagnose_and_repair_mesh(mesh):
    print("\n--- Mesh Diagnostics ---")
    # Check if the mesh is watertight
    if mesh.is_watertight:
        print("Trimesh: Mesh is watertight.")
    else:
        print("Trimesh: Mesh is not watertight. Attempting to fix...")
        mesh = mesh.fill_holes()
        if mesh.is_watertight:
            print("Trimesh: Mesh successfully made watertight.")
        else:
            print("Trimesh: Failed to make mesh watertight.")
    
    # Ensure consistent winding
    mesh.fix_normals()
    
    # Remove duplicate and degenerate faces
    mesh.remove_duplicate_faces()
    mesh.remove_degenerate_faces()
    
    # Merge close vertices
    mesh.merge_vertices()
    
    # Re-check manifoldness and watertightness
    print(f"Watertight after repairs: {mesh.is_watertight}")
    print(f"Winding consistent: {mesh.is_winding_consistent}")
    
    # Check for non-manifold edges
    try:
        non_manifold_edges = mesh.non_manifold_edges
        if len(non_manifold_edges) > 0:
            print(f"Trimesh: Mesh has {len(non_manifold_edges)} non-manifold edges.")
        else:
            print("Trimesh: Mesh has no non-manifold edges.")
    except AttributeError:
        print("Trimesh: 'non_manifold_edges' attribute not found. Using manual method.")
        # Manual method to detect non-manifold edges
        edges = mesh.faces[:, [0,1]]
        edges = np.sort(edges, axis=1)
        all_edges = np.vstack((edges, mesh.faces[:, [1,2]], mesh.faces[:, [2,0]]))
        unique_edges, counts = np.unique(all_edges, axis=0, return_counts=True)
        non_manifold_edges = unique_edges[counts > 2]
        if len(non_manifold_edges) > 0:
            print(f"Manual Check: Mesh has {len(non_manifold_edges)} non-manifold edges.")
        else:
            print("Manual Check: Mesh has no non-manifold edges.")
    
    return mesh, non_manifold_edges if 'non_manifold_edges' in locals() else np.array([])

# Diagnose and repair the mesh
mesh, non_manifold_edges = diagnose_and_repair_mesh(mesh)

# -------------------------------------------
# 6. Further Repairs with PyVista (If Necessary)
# -------------------------------------------
def further_repair_with_pyvista(mesh, non_manifold_edges):
    if not mesh.is_watertight or not mesh.is_winding_consistent or len(non_manifold_edges) > 0:
        print("\nPyVista: Performing additional mesh repairs...")
        # Convert Trimesh to PyVista PolyData
        faces_pv = np.hstack([np.full((mesh.faces.shape[0], 1), 3), mesh.faces]).astype(np.int32)
        mesh_pv = pv.PolyData(mesh.vertices, faces_pv)
        
        # Clean the mesh
        mesh_pv = mesh_pv.clean(tolerance=1e-5, inplace=False)
        
        # Remove any unused points
        mesh_pv.remove_unreferenced_points(inplace=True)
        
        # Re-check manifoldness
        if mesh_pv.is_manifold:
            print("PyVista: Mesh is manifold after cleaning.")
        else:
            print("PyVista: Mesh is still non-manifold after cleaning.")
        
        # Triangulate to ensure all faces are triangles
        if not mesh_pv.is_all_triangles():
            mesh_pv.triangulate(inplace=True)
        
        # Convert back to Trimesh
        mesh = trimesh.Trimesh(vertices=mesh_pv.points, faces=mesh_pv.faces.reshape(-1, 4)[:,1:4], process=True)
        
        # Final checks
        print(f"Final Watertight: {mesh.is_watertight}")
        print(f"Final Winding Consistent: {mesh.is_winding_consistent}")
        
        try:
            non_manifold_edges = mesh.non_manifold_edges
            if len(non_manifold_edges) > 0:
                print(f"Trimesh: Mesh has {len(non_manifold_edges)} non-manifold edges after PyVista repairs.")
            else:
                print("Trimesh: Mesh has no non-manifold edges after PyVista repairs.")
        except AttributeError:
            print("Trimesh: 'non_manifold_edges' attribute not found after PyVista repairs.")
            # Manual check again
            edges = mesh.faces[:, [0,1]]
            edges = np.sort(edges, axis=1)
            all_edges = np.vstack((edges, mesh.faces[:, [1,2]], mesh.faces[:, [2,0]]))
            unique_edges, counts = np.unique(all_edges, axis=0, return_counts=True)
            non_manifold_edges = unique_edges[counts > 2]
            if len(non_manifold_edges) > 0:
                print(f"Manual Check After PyVista: Mesh has {len(non_manifold_edges)} non-manifold edges.")
            else:
                print("Manual Check After PyVista: Mesh has no non-manifold edges.")
    
    else:
        print("\nNo further repairs needed with PyVista.")
    
    return mesh

# Further repair if necessary
mesh = further_repair_with_pyvista(mesh, non_manifold_edges)

# -------------------------------------------
# 7. Visualize the Repaired Mesh with Plotly (Optional)
# -------------------------------------------
def visualize_with_plotly(mesh):
    fig = go.Figure(data=[
        go.Mesh3d(
            x=mesh.vertices[:,0],
            y=mesh.vertices[:,1],
            z=mesh.vertices[:,2],
            i=mesh.faces[:,0],
            j=mesh.faces[:,1],
            k=mesh.faces[:,2],
            color='lightblue',
            opacity=0.5,
            name='Closed Surface',
            showscale=True,
            colorbar=dict(title="Surface Height (µm)")
        )
    ])
    
    fig.update_layout(
        title="3D Closed Rough Surface Mesh",
        scene=dict(
            xaxis_title="Width (µm)",
            yaxis_title="Height (µm)",
            zaxis_title="Surface Height (µm)",
            camera=dict(
                eye=dict(x=1.25, y=1.25, z=1.25)
            ),
            aspectratio=dict(x=1, y=1, z=0.5)
        ),
        margin=dict(l=0, r=0, b=0, t=50),
        height=700,
        width=800
    )
    
    fig.show()

# Visualize the mesh with Plotly
visualize_with_plotly(mesh)

# -------------------------------------------
# 8. Tetrahedralize the Repaired Mesh with TetGen
# -------------------------------------------
def tetrahedralize_with_tetgen(mesh):
    try:
        # Option 1: Pass vertices and faces directly
        tet = tetgen.TetGen(mesh.vertices, mesh.faces)
        tet.tetrahedralize(order=1, mindihedral=10, minratio=1.2, diagnose=True, quiet=False)  # Adjusted parameters
    except TypeError as e:
        print(f"TypeError during TetGen initialization: {e}")
        print("Attempting to pass a tuple of (points, faces) instead.")
        try:
            # Option 2: Pass as a tuple
            tet = tetgen.TetGen((mesh.vertices, mesh.faces))
            tet.tetrahedralize(order=1, mindihedral=10, minratio=1.2, diagnose=True, quiet=False)
        except Exception as e2:
            print(f"Failed to tetrahedralize with tuple input: {e2}")
            raise e2
    except RuntimeError as e:
        print(f"RuntimeError during TetGen tetrahedralization: {e}")
        print("Mesh may still have issues. Consider simplifying or further repairing the mesh.")
        raise e
    
    # Extract tetrahedral mesh
    tet_mesh = tet.grid
    
    # Verify that TetGen successfully tetrahedralized the mesh
    if tet_mesh is None:
        raise RuntimeError("TetGen failed to generate a tetrahedral mesh.")
    
    return tet_mesh

# Perform tetrahedralization
try:
    tet_mesh = tetrahedralize_with_tetgen(mesh)
except RuntimeError as e:
    print(f"Tetrahedralization failed: {e}")
    sys.exit(1)

# -------------------------------------------
# 9. Extract Points and Cells for Polyscope
# -------------------------------------------
def extract_points_and_cells(tet_mesh):
    points = tet_mesh.points
    
    # Extract cells: TetGen returns cells in a flat array where each tetrahedron starts with the number 4
    # (indicating a tetrahedron) followed by four point indices
    try:
        cells = tet_mesh.cells.reshape(-1, 5)[:,1:5]  # Each tetrahedron has 4 points
    except ValueError as ve:
        print(f"ValueError during cell reshaping: {ve}")
        print("Attempting to extract cells differently.")
        cells = tet_mesh.cells.reshape(-1, 5)[:,1:5]
    
    return points, cells

points, cells = extract_points_and_cells(tet_mesh)

# -------------------------------------------
# 10. Register the Tetrahedral Grid with Polyscope and Add Material Properties
# -------------------------------------------
def register_with_polyscope(points, cells):
    # Register the tetrahedral grid with Polyscope
    ps_mesh = ps.register_tetrahedral_grid(
        "TetMesh",
        points,
        cells
    )
    
    # Assign real material properties
    # Example: Steel properties
    young_modulus = 210e3  # MPa
    poissons_ratio = 0.3
    density = 7850  # kg/m³
    thermal_conductivity = 50  # W/(m·K)
    specific_heat = 470  # J/(kg·K)
    
    # Create arrays for material properties
    # Assigning uniform properties; modify as needed for varying properties
    E = np.full(points.shape[0], young_modulus)
    nu = np.full(points.shape[0], poissons_ratio)
    rho = np.full(points.shape[0], density)
    k = np.full(points.shape[0], thermal_conductivity)
    c = np.full(points.shape[0], specific_heat)
    
    # Add scalar quantities to Polyscope
    ps_mesh.add_scalar_quantity("Young's Modulus (MPa)", E, defined_on="vertices", cmap="viridis")
    ps_mesh.add_scalar_quantity("Poisson's Ratio", nu, defined_on="vertices", cmap="viridis")
    ps_mesh.add_scalar_quantity("Density (kg/m³)", rho, defined_on="vertices", cmap="viridis")
    ps_mesh.add_scalar_quantity("Thermal Conductivity (W/(m·K))", k, defined_on="vertices", cmap="plasma")
    ps_mesh.add_scalar_quantity("Specific Heat (J/(kg·K))", c, defined_on="vertices", cmap="inferno")
    
    # Optionally, add more material properties as needed
    # For example, adding a material identifier if multiple materials are present
    # material_id = np.ones(cells.shape[0], dtype=int)  # All tetrahedrons have material ID 1
    # ps_mesh.add_scalar_quantity("Material ID", material_id, defined_on="cells", cmap="jet")
    
    return ps_mesh

ps_mesh = register_with_polyscope(points, cells)

# -------------------------------------------
# 11. Show Polyscope Visualization
# -------------------------------------------
ps.show()

# -------------------------------------------
# 12. Export the Tetrahedral Mesh with Material Properties
# -------------------------------------------
def export_with_meshio(points, cells):
    # Define material properties per tetrahedron (optional)
    # For uniform materials, assign a single material ID
    material_id = np.ones(cells.shape[0], dtype=int)  # All tetrahedrons have material ID 1
    
    # Create cell data dictionary
    cell_data = {
        "material_id": [material_id]
    }
    
    # Create a meshio Mesh object
    tetra_cells = [("tetra", cells)]
    meshio_mesh = meshio.Mesh(points=points, cells=tetra_cells, cell_data=cell_data)
    
    # Export to .msh file (compatible with many FEA tools)
    meshio.write("heightmap_tet.msh", meshio_mesh)
    
    print("\nTetrahedral mesh with material properties exported to 'heightmap_tet.msh'.")

# Export the mesh
export_with_meshio(points, cells)


Trimesh version 4.4.9 is sufficient.

--- Mesh Diagnostics ---
Trimesh: Mesh is watertight.


C:\Users\antoi\AppData\Local\Temp\ipykernel_75992\1602207168.py:162: DeprecationWarning:

`remove_duplicate_faces` is deprecated and will be removed in March 2024: replace with `mesh.update_faces(mesh.unique_faces())`

C:\Users\antoi\AppData\Local\Temp\ipykernel_75992\1602207168.py:163: DeprecationWarning:

`remove_degenerate_faces` is deprecated and will be removed in March 2024 replace with `self.update_faces(self.nondegenerate_faces(height=height))`



Watertight after repairs: True
Winding consistent: True
Trimesh: 'non_manifold_edges' attribute not found. Using manual method.
Manual Check: Mesh has no non-manifold edges.

No further repairs needed with PyVista.


: 